## 1. Импорт пакетов и файла

In [1]:
%matplotlib inline
import pandas as pd
import string
import re
import nltk
import pymorphy3

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('pdf_files.csv').drop('Unnamed: 0', axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        11 non-null     object 
 1   description  11 non-null     object 
 2   date         11 non-null     object 
 3   tags         11 non-null     object 
 4   rating       11 non-null     float64
dtypes: float64(1), object(4)
memory usage: 572.0+ bytes


In [4]:
df.head()

,title,description,date,tags,rating
0,"Cocoapods, Carthage, SPM: как выбрать менеджер...",117.94 Рейтинг red_mad_robot №1 в разработке ц...,6 часов назад,"Хабы: Блог компании red_mad_robot , Разработ...",117.94
1,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y, Информационная б...",71.07
2,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y , Информационная...",71.07
3,Быстрое начало работы с Gitlab CI/CD: пайплайн...,4.29 Оценка 280.79 Рейтинг Southbridge Обеспеч...,2 часа назад,"Хабы: Блог компании Southbridge , Тестирован...",280.79
4,Генеральный директор Mozilla покинула свой пост,"Митчелл Бейкер, гендиректор Mozilla с 2020 год...",12 минут назад,"Хабы: IT-компании, Управление персоналом, Кар...",145.10


## 2. Предварительная обработка данных

In [5]:
import re

def remove_english_words(text):
    # Удаляет слова, состоящие только из английских букв (включая сокращения)
    return re.sub(r'\b[a-zA-Z]+\b', '', text)

def remove_punctuation(text): 
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text): 
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text): 
    return re.sub(r'\s+', ' ', text, flags=re.I)

st = '❯\xa0—«»'
def remove_othersymbol(text):
    return ''.join([ch if ch not in st else ' ' for ch in text])

In [6]:
df['prep_text'] = [remove_english_words(remove_multiple_spaces(remove_numbers(remove_othersymbol(remove_punctuation(text.lower()))))) for text in df['description']]

In [7]:
df['description'][0]

'117.94 Рейтинг red_mad_robot №1 в разработке цифровых решений для бизнеса  6 часов назад   Cocoapods, Carthage, SPM: как выбрать менеджер зависимостей в iOS 179 Блог компании red_mad_robot ,  Разработка под iOS* FAQ  Старший iOS-разработчик Аня Кочешкова рассказывает, чем отличаются три менеджера зависимостей, в каких случаях и для каких задач подойдёт тот или иной. Материал будет полезен джун-специалистам, которые только начали погружаться в разработку: специально для них подробно объясняем, что такое семантическое версионирование, как устроены модули кода и в чём разница между динамическими и статическими библиотеками. Что такое менеджер зависимостей и зачем он нужен В современной разработке зависимость — это написанный кем-то другим код, который используется в вашей программе. Добавление зависимости позволяет воспользоваться уже сделанной кем-то хорошей работой и избежать «изобретения велосипеда», а также лишних действий: проектирования, написания, тестирования, отладки и поддержки

In [8]:
df['prep_text'][0]

' рейтинг    № в разработке цифровых решений для бизнеса часов назад    как выбрать менеджер зависимостей в  блог компании    разработка под   старший  разработчик аня кочешкова рассказывает чем отличаются три менеджера зависимостей в каких случаях и для каких задач подойдёт тот или иной материал будет полезен джун специалистам которые только начали погружаться в разработку специально для них подробно объясняем что такое семантическое версионирование как устроены модули кода и в чём разница между динамическими и статическими библиотеками что такое менеджер зависимостей и зачем он нужен в современной разработке зависимость это написанный кем то другим код который используется в вашей программе добавление зависимости позволяет воспользоваться уже сделанной кем то хорошей работой и избежать изобретения велосипеда а также лишних действий проектирования написания тестирования отладки и поддержки определённой логики такие куски логики называют пакетами библиотеками или модулями чем сложнее п

In [9]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 
    'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 
    'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 
    'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 
    'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'жизнь', 
    'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 
    'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'другой', 
    'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 
    'моя', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 
    'между', 'это', 'пока', 'об', 'часто', 'теперь', 'слишком', 'либо','хабр', 'хабрахабр', 'habr', 'комментарий', 'комментарии', 'статья', 'статьи', 'автор', 'пользователь', 'пост', 'блог',
    'читать', 'прочитать', 'написать', 'написал', 'подробнее', 'рейтинг', 'плюс', 'минус', 'карма', 'хаб', 'хабы',
    'git', 'github', 'пример', 'код', 'кода', 'коде', 'коду', 'кодом', 'python', 'javascript', 'программа', 'проект', 
    'использовать', 'использование', 'файл', 'файлы', 'сделать', 'создать', 'решение', 'задача', 'задачи', 'работа', 
    'примеры', 'документация', 'класс', 'функция', 'метод', 'сервер', 'клиент', 'данные', 'данных', 'база', 'базы',
    'http', 'https', 'www', 'com', 'ru', 'ссылка', 'img', 'src', 'div', 'класса', 'alt', 'nbsp', 'quot', 'lt', 'gt', 
    'является', 'являются', 'можно', 'найти', 'стоит', 'хочу', 'хотел', 'кажется', 'получить', 'помощью',
    'сегодня', 'вчера', 'неделя', 'месяц', 'год', 'года', 'лет', 'время', 'раз', 'версия'])

In [10]:
def tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [11]:
df['tokenize_text'] = [tokenize(text) for text in df['prep_text']]

In [12]:
stemmer = SnowballStemmer("russian")

stem_list = []
for text in (df['tokenize_text']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = stemmer.stem(word)
            res.append(p)
        text = " ".join(res)
        stem_list.append(text)
    except Exception as e:
        print(e)
        
df['text_stem'] = stem_list

In [13]:
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [14]:
%%time
lemm_texts_list = []
for text in (df['tokenize_text']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = morph.parse(word)[0]
            res.append(p.normal_form)
        text = " ".join(res)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm'] = lemm_texts_list

CPU times: total: 1 s
Wall time: 1.05 s


In [15]:
df['text_lemm'] = [tokenize(text) for text in df['text_lemm']]

In [182]:
df.head(20)

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
0,"Cocoapods, Carthage, SPM: как выбрать менеджер...",117.94 Рейтинг red_mad_robot №1 в разработке ц...,6 часов назад,"Хабы: Блог компании red_mad_robot , Разработ...",117.94,рейтинг № в разработке цифровых решений дл...,№ разработке цифровых решений бизнеса часов на...,№ разработк цифров решен бизнес час назад выбр...,№ разработка цифровой бизнес час назад выбрать...,0
1,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y, Информационная б...",71.07,рейтинг корпоративный облачный провайдер ав...,корпоративный облачный провайдер оригинала взл...,корпоративн облачн провайдер оригина взлом ‑ак...,корпоративный облачный провайдер оригинал взло...,0
2,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y , Информационная...",71.07,рейтинг корпоративный облачный провайдер ав...,корпоративный облачный провайдер оригинала взл...,корпоративн облачн провайдер оригина взлом акк...,корпоративный облачный провайдер оригинал взло...,0
3,Быстрое начало работы с Gitlab CI/CD: пайплайн...,4.29 Оценка 280.79 Рейтинг Southbridge Обеспеч...,2 часа назад,"Хабы: Блог компании Southbridge , Тестирован...",280.79,оценка рейтинг обеспечиваем стабильную работ...,оценка обеспечиваем стабильную работу проектов...,оценк обеспечива стабильн работ проект оригина...,оценка обеспечивать стабильный оригинал переве...,2
4,Генеральный директор Mozilla покинула свой пост,"Митчелл Бейкер, гендиректор Mozilla с 2020 год...",12 минут назад,"Хабы: IT-компании, Управление персоналом, Кар...",145.10,митчелл бейкер гендиректор с года объявила чт...,митчелл бейкер гендиректор объявила покидает с...,митчелл бейкер гендиректор объяв покида сво во...,митчелла бейкер гендиректор объявить покидать ...,3
5,ИИ-агенты в Альфа-Банке: нейросети создают авт...,404.32 Рейтинг Альфа-Банк Лучший мобильный бан...,2 часа назад,"Хабы: Блог компании Альфа-Банк, Искусственный ...",404.32,рейтинг альфа банк лучший мобильный банк по в...,альфа банк лучший мобильный банк версии подпис...,альф банк лучш мобильн банк верс подписа альф ...,альфа банк хороший мобильный банк подписаться ...,0
6,"Инструменты наблюдаемости, о которых нужно зна...",2376.9 Рейтинг Автор оригинала: Lahiru Hewaw...,4 часа назад,"Хабы: Блог компании , Open source , Хранени...",2376.90,рейтинг автор оригинала часа назад инструме...,оригинала часа назад инструменты наблюдаемости...,оригина час назад инструмент наблюдаем котор н...,оригинал час назад инструмент наблюдаемость ко...,0
7,История российской науки: напишем вместе? 1.3K,4.58 Оценка 353.18 Рейтинг Хабр Экосистема для...,10 часов назад,"Хабы: Блог компании Хабр, Научно-популярное",353.18,оценка рейтинг хабр экосистема для развития л...,оценка экосистема развития людей вовлеченных ф...,оценк экосистем развит люд вовлечен феврал ден...,оценка экосистема развитие человек вовлечь фев...,0
8,Как создать аппаратный эмулятор CD-ROM без пая...,"2394.92 Рейтинг Несмотря на то, что постепен...",20 мар в 14:00,"Хабы: Блог компании , Системное администриров...",2394.92,рейтинг несмотря на то что постепенно оптичес...,несмотря постепенно оптические диски уходят пр...,несмотр постепен оптическ диск уход прошл обра...,несмотря постепенно оптический диск уходить пр...,1
9,Может ли chatGPT забронировать столик в рестор...,Идея А почему бы не использовать возможности ...,6 часов назад,"Хабы: Мессенджеры , Python , Искусственный и...",14.00,идея а почему бы не использовать возможности ...,идея почему возможности попросить делать напри...,иде поч возможн попрос дела например дава попр...,идея почему возможность попросить делать напри...,4


## Векторизация

In [118]:
tfidf_vectorizer = TfidfVectorizer(max_df=1, max_features=100000000,
                                 min_df=1, stop_words=russian_stopwords,
                                 ngram_range=(1,3))

In [119]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_lemm'])

In [120]:
tfidf_matrix.shape

(11, 16302)

## Тематическое моделирование

In [121]:
from sklearn.decomposition import TruncatedSVD

In [122]:
# создание модели LSA
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 1: пайплайна, задание, инстанс, непрерывный, концепция, менеджер зависимость, доставка, определимый, среда, этап пайплайна
Topic 2: учёный, наука, пайплайна, российский наука, ipадрес имя пароль, ipадрес, вконтакте корпоративный облачный, представитель настройка язык, россия представитель настройка, сеть сообщесть обзор
Topic 3: пайплайна, новый процессор, задание, альфа банк, альфа, выпустить, эмулятор, диск, режим, инстанс
Topic 4: трассировка, метрика, пайплайна, первопричина, визуализация, алерот, выявление, анализ первопричина, производительность приложение, популярный способ
Topic 5: пайплайна, онлайн, бейкер, учёный, календарь, генеральный директор, февраль онлайн календарь, онлайн календарь, февраль онлайн, должность


In [123]:
from sklearn.decomposition import NMF

In [124]:
# создание модели NMF
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 1: пайплайна, задание, инстанс, непрерывный, концепция, тый вроде, цель безопасность тый, сообщество обзор перевод, сообщество обзор, безопасность тый вроде
Topic 2: трассировка, эмулятор, метрика, диск, режим, контроллер, ядро, визуализация, первопричина, оптический
Topic 3: учёный, наука, российский наука, пирог, корсак, булат, павел, атомный, медицина, век
Topic 4: новый процессор, выпустить, новшество, привычный, разработка софт, разработка софт искусственный, скоро, панель, упоминать, активно
Topic 5: альфа, альфа банк, онлайн, бейкер, менеджер зависимость, ан, ан заря, заря, автотест, генеральный директор


## Кластеризация

In [125]:
num_clusters = 5

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, random_state=0)

In [126]:
km.fit(tfidf_matrix)

KMeans(n_clusters=5, random_state=0)

In [127]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [128]:
clusterkm = km.labels_.tolist()
df['cluster'] = clusterkm

In [129]:
df['cluster'].value_counts()

cluster
0    7
2    1
3    1
1    1
4    1
Name: count, dtype: int64

In [130]:
df[df['cluster']==0]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
0,"Cocoapods, Carthage, SPM: как выбрать менеджер...",117.94 Рейтинг red_mad_robot №1 в разработке ц...,6 часов назад,"Хабы: Блог компании red_mad_robot , Разработ...",117.94,рейтинг № в разработке цифровых решений дл...,№ разработке цифровых решений бизнеса часов на...,№ разработк цифров решен бизнес час назад выбр...,№ разработка цифровой бизнес час назад выбрать...,0
1,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y, Информационная б...",71.07,рейтинг корпоративный облачный провайдер ав...,корпоративный облачный провайдер оригинала взл...,корпоративн облачн провайдер оригина взлом ‑ак...,корпоративный облачный провайдер оригинал взло...,0
2,OSINT & Hacking — как работает фишинг для нель...,71.07 Рейтинг Cloud4Y #1 Корпоративный облачны...,5 часов назад,"Хабы: Блог компании Cloud4Y , Информационная...",71.07,рейтинг корпоративный облачный провайдер ав...,корпоративный облачный провайдер оригинала взл...,корпоративн облачн провайдер оригина взлом акк...,корпоративный облачный провайдер оригинал взло...,0
5,ИИ-агенты в Альфа-Банке: нейросети создают авт...,404.32 Рейтинг Альфа-Банк Лучший мобильный бан...,2 часа назад,"Хабы: Блог компании Альфа-Банк, Искусственный ...",404.32,рейтинг альфа банк лучший мобильный банк по в...,альфа банк лучший мобильный банк версии подпис...,альф банк лучш мобильн банк верс подписа альф ...,альфа банк хороший мобильный банк подписаться ...,0
6,"Инструменты наблюдаемости, о которых нужно зна...",2376.9 Рейтинг Автор оригинала: Lahiru Hewaw...,4 часа назад,"Хабы: Блог компании , Open source , Хранени...",2376.90,рейтинг автор оригинала часа назад инструме...,оригинала часа назад инструменты наблюдаемости...,оригина час назад инструмент наблюдаем котор н...,оригинал час назад инструмент наблюдаемость ко...,0
7,История российской науки: напишем вместе? 1.3K,4.58 Оценка 353.18 Рейтинг Хабр Экосистема для...,10 часов назад,"Хабы: Блог компании Хабр, Научно-популярное",353.18,оценка рейтинг хабр экосистема для развития л...,оценка экосистема развития людей вовлеченных ф...,оценк экосистем развит люд вовлечен феврал ден...,оценка экосистема развитие человек вовлечь фев...,0
10,Новые утечки. Что мы знаем о выходе Windows 12,4.74 Оценка 414.35 Рейтинг getmatch Рассказыва...,4 часа назад,"Хабы: Блог компании getmatch , Разработка по...",414.35,оценка рейтинг рассказываем о том как строит...,оценка рассказываем строить карьеру весь интер...,оценк рассказыва стро карьер ве интернет готов...,оценка рассказывать строить карьера весь интер...,0


In [140]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==0]['text_lemm'])
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель NMF:
Topic 1: лог, трассировка, мониторинг, метрика, первопричина, визуализация, производительность, выполнять, наблюдаемость, алерот
Topic 2: библиотека, указать, менеджер, менеджер зависимость, российский, учёный, динамический, спечь, наука, российский наука
Topic 3: банк, альфа, альфа банк, спасибо внимание корпоративный, похожий сайт, похожий сайт ваш, представитель информация, песочница дата, песочница дата регистрация, сайт ваш аккаунт
Topic 4: вконтакте корпоративный облачный, внимание ещё, например ipадрес имя, чтоть, чтоть вроде всё, безопасность чтоть, безопасность чтоть вроде, внимание ещё интересный, вконтакте корпоративный, информация например ipадрес
Topic 5: процессор, ос, утечка, новый процессор, возможно, выпустить, экран, последний, софт, скоро


In [141]:
df[df['cluster']==1]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
8,Как создать аппаратный эмулятор CD-ROM без пая...,"2394.92 Рейтинг Несмотря на то, что постепен...",20 мар в 14:00,"Хабы: Блог компании , Системное администриров...",2394.92,рейтинг несмотря на то что постепенно оптичес...,несмотря постепенно оптические диски уходят пр...,несмотр постепен оптическ диск уход прошл обра...,несмотря постепенно оптический диск уходить пр...,1


In [142]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==1]['text_lemm'])
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель NMF:
Topic 1: устройство, который, образ, система, команда, эмулятор, диск, компьютер, режим, работать
Topic 2: никакой, запуск, канал, запускать, сложный, оказываться, назад удивительный, условие образ эмулироваться, установить пароль, телефон играть роль
Topic 3: наш эмулятор, радиоканал, доступ случай операционный, момент многий дистрибутив, понимание заинтересовать мочь, разношёрстный парк старый, необходимо наличие, передавать радиоканал, перекомпиляция ядро изменение, прошлое образ диск
Topic 4: низкий цена, поддержка эмуляция, устройство который последовательный, ядро кросс компиляция, объём тема, система понадобиться ещё, память перезагрузка, необходимо контроллер мочь, доступный профиль, запускать телефон
Topic 5: активировать который отключить, сократить компиляция больший, понять смочь, канал канал, который содержать реально, услуга корпоративный медийный, ассоциировать образ, режим, выбор драйвер, стек именно


In [143]:
df[df['cluster']==2]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
3,Быстрое начало работы с Gitlab CI/CD: пайплайн...,4.29 Оценка 280.79 Рейтинг Southbridge Обеспеч...,2 часа назад,"Хабы: Блог компании Southbridge , Тестирован...",280.79,оценка рейтинг обеспечиваем стабильную работ...,оценка обеспечиваем стабильную работу проектов...,оценк обеспечива стабильн работ проект оригина...,оценка обеспечивать стабильный оригинал переве...,2


In [144]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==2]['text_lemm'])
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель NMF:
Topic 1: пайплайна, этап, который, команда, несколько, задание, сайт, компания, разработчик, настройка
Topic 2: февраль численность человек, уменьшение количество бутылочный, команда каждый который, тестирование невозможно, шаг позволять гарантировать, основный ветка, дата регистрация ноябрь, процесс который происходить, непрерывный, ноябрь дата основание
Topic 3: задание задание, сайт компания, автоматический, инструмент, некоторый, повышение, системный администрирование туториал, разработка уменьшение количество, корпоративный медийный реклама, создание пайплайна развёртывание
Topic 4: непрерывный интеграция, системный администрирование, интеграция, статический веб, несколько раннер, шаг, сборка, этап, случай, способность человек стимулировать
Topic 5: переполненный ами, полный список смотреть, преимущество бизнес, сотрудник, изменение автоматически развёртываться, пайплайна обычно, постоянно готовый деплоить, предположим хотеть простой, исходный, кодовый постоянно готовы

In [136]:
df[df['cluster']==3]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
4,Генеральный директор Mozilla покинула свой пост,"Митчелл Бейкер, гендиректор Mozilla с 2020 год...",12 минут назад,"Хабы: IT-компании, Управление персоналом, Кар...",145.1,митчелл бейкер гендиректор с года объявила чт...,митчелл бейкер гендиректор объявила покидает с...,митчелл бейкер гендиректор объяв покида сво во...,митчелла бейкер гендиректор объявить покидать ...,3


In [145]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==3]['text_lemm'])
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель NMF:
Topic 1: назад, час, час назад, компания, ответ, реклама, мина, директор, мина час, новый
Topic 2: оформить, лор чемберс, час назад, свет час, дипфейк, принять первый чтение, продвигать миссия, почему работать слайдер, обучение, внимание
Topic 3: принтер мина, хабра реклама, последний работать, цена, сберпремьер организация акция, планировать беспилотный, регулировать кислотность, качество морской, российский банк, аккаунт войти
Topic 4: живучий новый, генеральный директор конец, совет директор течение, скидочный снегопад, развитие, английский работать сила, соглашение конфиденциальность услуга, бизнес, требовать, уничтожение техника запретить
Topic 5: достигнуть беспрецедентный, сообщение митчелла работать, присылать, модель ответ делать, устройство сайт, банк вернуть, состав совет, компания создание платформа, станция, пять создавать аналогичный


In [146]:
df[df['cluster']==4]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
9,Может ли chatGPT забронировать столик в рестор...,Идея А почему бы не использовать возможности ...,6 часов назад,"Хабы: Мессенджеры , Python , Искусственный и...",14.0,идея а почему бы не использовать возможности ...,идея почему возможности попросить делать напри...,иде поч возможн попрос дела например дава попр...,идея почему возможность попросить делать напри...,4


In [148]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==4]['text_lemm'])
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель NMF:
Topic 1: квартира, заря, ан, ан заря, вопрос, ваш, мочь, сказать, ресторан, информация
Topic 2: пожалуйста, задавать, передать информация, ответить вопрос, попробовать, страховой, день, непосредственный, хозяин, информация рассматривать ипотечный
Topic 3: квартира, информация ответить ваш, андрей первый, компания песочница, компания песочница информация, деньга, документ, искать, агентство ан, миллион
Topic 4: ключ час, столик ресторан понадобиться, наличие содзаесть, начинать задавать вопрос, ответить ваш, ход тестирование, который попробовать, сбер втб, вместе запустить, рестор ресторан твой
Topic 5: узнать продажа, регистрироваться сайт получать, интеграция предположим, нужно, втб спасибо информация, справка наличие, поддержка, условие ставка каждый, компания документ, интеллект получение ключ


In [149]:
# Восстанавливаем настройки
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

## Классификация

In [150]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text_lemm'], df['cluster'], 
                                                      test_size=0.3, 
                                                      random_state=0)

In [151]:
len(X_train)

7

In [152]:
len(X_test)

4

In [153]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [154]:
vectorizer = TfidfVectorizer(max_features=10000000, ngram_range=(1,3), stop_words=russian_stopwords)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## LogisticRegression

In [155]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

LogisticRegression()

In [156]:
y_pred = model_lr.predict(X_test_tfidf)

In [157]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.50         4
   macro avg       0.17      0.33      0.22         4
weighted avg       0.25      0.50      0.33         4



In [158]:
X_test_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [159]:
y_pred

array([0, 0, 0, 0], dtype=int64)

## RandomForestClassifier

In [160]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [161]:
y_pred = model_rf.predict(X_test_tfidf)

In [162]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.50         4
   macro avg       0.17      0.33      0.22         4
weighted avg       0.25      0.50      0.33         4



## KNeighborsClassifier

In [163]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier()

In [164]:
y_pred = model_knn.predict(X_test_tfidf)

In [165]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.50         4
   macro avg       0.17      0.33      0.22         4
weighted avg       0.25      0.50      0.33         4



## Сохранение моделей

In [166]:
import pickle

with open('model_knn_pdf_files.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [167]:
with open('vectorizer_pdf_files.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [168]:
df.to_csv('data_pdf_files.csv')

In [169]:
import pickle
import joblib
from catboost import CatBoostClassifier
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer

In [170]:
with open('model_knn_pdf_files.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer_pdf_files.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

In [171]:
def fun_punctuation_text(text):
    text = text.lower()
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    text = ''.join([i if not i.isdigit() else '' for i in text])
    text = ''.join([i if i.isalpha() else ' ' for i in text])
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub('[a-z]', '', text, flags=re.I)
    st = '❯\xa0'
    text = ''.join([ch if ch not in st else ' ' for ch in text])
    return text

In [172]:
def fun_lemmatizing_text(text):
    tokens = word_tokenize(text)
    res = list()
    for word in tokens:
        p = pymorphy3.MorphAnalyzer(lang='ru').parse(word)[0]
        res.append(p.normal_form)  
    text = " ".join(res)
    return text

In [173]:
def fun_tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [174]:
t1 = "Криптовалюта - это такая штука, которую, честно говоря, невероятно сложно объяснить. В этой статье я попробую сделать крипту понятнее на простом доступном языке. Еще недавно, если вы не жили в Кремниевой долине, можно было долго не слышать о криптовалюте. Но вдруг стало невозможно избежать того, чтобы о ней не говорили даже первоклашки. Криптовалюта! Несколько лет назад это казалось хайповой технологией, которую большинство людей могли спокойно проигнорировать, как гироскутеры или очки виртуальной реальности. Но ее экономическая и культурная власть стала настолько велика, чтобы ее невозможно не замечать. По данным недавнего опроса двадцать процентов взрослых людей и 36 процентов миллениалов владеют криптовалютой. Криптовалютные приложения для торговли входят в топы магазинов App Store или Google Play. Сегодня рынок криптовалют оценивается примерно в 1,75 триллиона долларов - примерно размер Google. Сотрудники крупных компаний и топ-менеджеры с горящими глазами бросают свои удобные работы, чтобы присоединиться к золотой лихорадке криптовалюты. По мере того как криптовалюта становится все более популярной, она вызывает жаркие споры. Ее страстные поклонники уверены, что она спасает мир, в то время как критики убеждены, что это все обман - пузырь спекуляции, уничтожающий окружающую среду, устроенный мошенниками и проданный алчным наивным людям, который, когда лопнет, вероятно, обрушит всю экономику.Я согласен со скептиками, что большая часть рынка криптовалют в настоящий момент состоит из переоцененных и, возможно, мошеннических активов, и я не проникся самыми утопическими чувствами, которыми делятся поклонники криптовалюты.Но по мере того как я все больше соприкасался по роду своей работы с криптовалютой - я пришел к выводу, что это не просто циничная попытка заработать деньги, и что действительно создаются вещи, имеющие реальную ценность. Я также понял, что когда столько денег, энергии и таланта идет на эту технологию, это, как минимум разумно уделять ей внимание, независимо от своих взглядов на криптовалюты.Однако самое крепкое мое убеждение о криптовалютах заключается в том, что они имеют просто ужасное объяснение о том, что это такое и как это работает.Последние два года я читал все, что мог о криптовалюте. Но я обнаружил, что большинство руководств для начинающих представляют собой скучные подкасты, мало исследованные видео на YouTube и блог-посты, написанные предвзятыми инвесторами. Многие антикрипто-аргументы, с другой стороны, ослабевали из-за неточностей и устаревших доводов, таких как утверждение, что криптовалюта полезна для преступников, несмотря на растущие доказательства того, что отслеживаемые реестры криптовалют делают её плохим выбором для незаконной деятельности.Мне не удалось найти трезвое, беспристрастное объяснение того, что же такое криптовалюта на самом деле - как она работает, кому она предназначена, что здесь на кону, где проходят линии боя - а также ответы на некоторые из самых распространенных вопросов, которые она вызывает.Это статья - на самом деле, раздел часто задаваемых вопросов, - попытка исправить ситуацию. Я постараюсь объяснить основные концепции как можно более понятно, отвечая на вопросы о криптовалютах.Поклонники криптовалюты, скорее всего, будут спорить с моими объяснениями, а убежденные противники могут считать их слишком снисходительными. Это нормально. Моя цель - не убедить вас, что криптовалюта хороша или плоха, что ее нужно запретить или превозносить, или что инвестиции в нее сделают вас богатыми или обанкротят. Моя цель - просто немного разъяснить что это такое. КРИПТОВАЛЮТЫ МОГУТ БЫТЬ РАЗРУШИТЕЛЬНЫМИ.Вторая причина, почему стоит обратить внимание на криптовалюты, заключается в том, что понимание их сегодня – лучший способ предотвратить их разрушительное воздействие в будущем.В начале 2010-х самым распространенным возражением против социальных сетей, таких как Facebook и Twitter, было то, что они просто не могут работать как бизнес. Эксперты предсказывали, что пользователи в конечном итоге устанут от фотографий с отпуска своих друзей, что рекламодатели уйдут и что вся индустрия социальных сетей рухнет. Теория заключалась не столько в том, что социальные медиа опасны или плохи; скорее, они были скучными и неинтересными, модным трендом, проходящим так быстро, как появился.То, о чем тогда никто не задавал вопросов - по крайней мере, не громко - это: А что, если социальные сети на самом деле станут безумно успешными? Какое регулирования должно существовать в мире, где Facebook и Twitter являются доминирующими платформами коммуникации? Как технологическим компаниям с миллиардами пользователей уравновесить свободу слова и безопасность? Какие функции продукта могут предотвратить трансформацию онлайн-ненависти и дезинформации в реальное насилие?К середине десятилетия, когда стало ясно, что это основные вопросы которые надо было решать, было уже слишком поздно. Механика платформ и бизнес-модели на основе рекламы уже были запущены, а скептики - которые могли бы направить эти приложения в лучшем направлении, если бы они относились к ним серьезнее с самого начала - были вынуждены затыкать дырки и проблемы и бежали за уезжающим поездом. Допускаем ли мы ту же ошибку с криптовалютой сегодня? Возможно. Еще никто не знает, будет ли криптовалюта 'работать' в наиболее широком смысле слова. (Тот, кто утверждает, что знает, скорее всего пытается вам что-то продать.) Но в ней есть реальные деньги и энергия, и многие ветераны технологической сферы, говорят, что сегодняшняя сцена криптовалют для них напоминает 2010 год - с тем лишь отличием, что технологии на этот раз разрушают финансовый сектор, а не медиа.Есть вероятность, что они ошибаются. Но если они правы - даже частично - лучшее время, чтобы начать обращать внимание на крипту, это сейчас.Третья причина, чтобы изучать криптовалюты, в том, что они могут быть по-настоящему увлекательными.Конечно, в них много глупого, сомнительного или отталкивающего. Но если вы сможете пройти мимо кричащих шарлатанов и прочитать запутанный жаргон, вы найдете бездонный колодец необычных, интересных и заставляющих задуматься проектов. Повестка дня криптовалют настолько огромна и многогранна - объединяя элементы экономики, инженерии, философии, права, искусства, энергетической политики и многое другое - она предлагает множество отправных точек для начинающих энтузиастов. Хотите обсудить влияние мировой экономики на развитие Bitcoin? Вероятно, для этого есть сервер Discord. Хотите присоединиться к DAO, которое инвестирует в NFT, или играть в видеоигру, которая платит вам криптовалютными токенами за победу? Вперед и с песней!КРИПТОВАЛЮТА – ЭТО УНИВЕРСАЛЬНЫЙ КЛЮЧ НОВОГО ПОКОЛЕНИЯ.Опросы показывают, что большУю долю владельцев криптовалют составляют богатые люди. При этом, существуют правые максималисты Bitcoin, которые верят, что криптовалюта освободит их от государственного тирании; левые фанаты Ethereum, которые хотят свергнуть большие банки; и спекулянты без идеологической привязанности, которые просто хотят получить прибыль и уйти. Эти сообщества постоянно ссорятся между собой, и у многих совершенно разные идеи о том, чем должна быть криптовалюта. Это делает их изучение увлекательным, особенно если вы сможете сохранить свой разум холодным.И если вы изучите основы криптовалюты, вам может открыться целый мир. Вы поймете, почему знаменитости меняют свои аватары в Twitter на мультяшных обезьян, и почему самый богатый человек в мире, Илон Маск, большую часть прошлого года твитил о цифровой валюте, названной в честь собаки. Криптовалюта может также стать своеобразным универсальным ключом нового поколения — возможно, это самый быстрый способ обновить ваше культурное сознание и разгадать образ мысли и поведение современной молодежи. После этих объяснений вы можете стать истинным верующим криптаном, убежденным скептиком или что-то между тем и другим. Принимайте решения самостоятельно! Все, что мне нужно, это ваше понимание технологии.Давай начнём с самого начала: что такое крипта?Всего десятилетие или два назад это слово обычно использовалось в качестве сокращения от криптография. Однако в последние годы оно стало более тесно связано с криптовалютами. Сегодня под 'крипто' обычно подразумевается целая вселенная технологий, связанных с блокчейнами – распределенными системами учета, которые обеспечивают работу цифровых валют вроде биткоина, но также служат базовым слоем технологии для таких вещей, как NFT, приложения web3 и торговые протоколы DeFi.Тут вы можете меня спросить - Ага, блокчейны! Напомнишь, не вдаваясь в слишком большие технические подробности, что это такое?В самом простом виде блокчейны – это общие базы данных, которые хранят и проверяют информацию способом, гарантирующим криптографическую безопасность.Можно представить блокчейн как таблицу Эксель, только вместо того чтобы быть размещенной в виде файла на вашем компьютере, блокчейны поддерживаются сетью компьютеров по всему миру. Эти компьютеры (иногда их называют майнерами или валидаторами) отвечают за хранение своих собственных копий базы данных, добавление и проверку новых записей, а также защиту базы данных от хакеров.Итак, блокчейны... это типа крутые облачные таблицы Эксель.Примерно так. Но есть как минимум три важные концептуальные различия.Во-первых, блокчейн децентрализован. Ему не нужен единственный владелец, который будет хранить этот файл у себя. Все это делается компьютерами в сети, используя то, что называется механизмом консенсуса — сложный алгоритм, который позволяет им договариваться о содержимом базы данных без необходимости нейтрального арбитра. Это делает блокчейны более безопасными, чем традиционные системы учета, уверены сторонники, так как ни один человек или компания не могут уничтожить блокчейн или изменить его содержимое, и любой, кто попытается взломать или изменить записи в журнале, должен будет взломать множество компьютеров одновременно.Второй важной особенностью блокчейнов является то, что они обычно публичны и с открытым исходным кодом, что означает, что в отличие файла Эксель на вашем компьютере, любой может проверить код публичного блокчейна или увидеть запись о любой транзакции. В-третьих, блокчейны обычно доступны только для добавления записей и не могут быть удалены, то есть, в отличие от таблицы Эксель, данные, добавленные в блокчейн, обычно являются постоянными и неизменными. И тут появляется закономерный вопрос: как связаны блокчейны и криптовалюты?Блокчейны впервые появились в 2009 году, когда программист под псевдонимом Сатоши Накамото опубликовал техническую документацию по Биткоину, первой в истории криптовалюте.Биткоин использовал блокчейн для отслеживания транзакций. Это было заметно, потому что впервые люди могли отправлять и получать деньги через интернет, не прибегая к услугам центральной власти, такой как банк или услугам по денежным переводам.Многие блокчейны до сих пор осуществляют транзакции с криптовалютами, и сейчас существует больше 10 000 различных криптовалют, согласно CoinMarketCap. Но многие блокчейны могут использоваться для хранения и других видов информации, включая NFT, фрагменты самоисполняющегося кода, известные как смарт-контракты, и полноценные приложения - и все это без необходимости центральной власти.Хорошо, но можем ли мы вернуться на секунду назад? Разве технические специалисты не говорили нам годы назад, что крипта - это новая и волнующая форма денег? И тем не менее, никто из моих знакомых не платит за аренду или не покупает продукты на Биткоины. Значит ли это, что эти люди просто ... ошибались?Это хороший вопрос. Действительно, сегодня практически никто не платит за вещи криптовалютой. Отчасти это связано с тем, что большинство продавцов до сих пор не принимают платежи в криптовалютах, и большие транзакционные комиссии могут делать непрактичным тратить небольшие суммы криптовалюты на повседневные расходы. Кроме того, стоимость популярных криптовалют, таких как Биткоин и Эфир, исторически росла, что делает их использование для офлайн-покупок несколько рискованным. (Обычно сожалеют о тех, кто использовал их в прошлом, например, о человеке, который в 2010 году купил две пиццы в Papa John's за Биткоины, которые тогда стоили около 40 долларов, но сейчас стоили бы около 400 миллионов долларов.)Также верно, что стоимость криптовалют резко выросла с начала эпохи Биткоина, несмотря на то, что они не являются основной валютой для ежедневных расходов большинства людей.Часть этого роста - это спекуляции - люди покупают криптоактивы в надежде продать их дороже впоследствии. Часть этого связана с тем, что блокчейны, появившиеся после Биткоина, такие как Ethereum и Solana, расширили возможности использования этой технологии.И некоторые любители крипты верят, что цены на такие криптовалюты, как Биткоин, в конечном итоге стабилизируются, что может сделать их более полезными в качестве средства платежа.Какие же есть реальные применения крипты, помимо финансовой спекуляции?В настоящее время многие успешные применения технологии крипты связаны с финансами или близкими к финансам областями. Например, люди используют крипту для перевода денег за границу своим родственникам, а банки используют блокчейны для проведения международных транзакций.Бум крипты также привел к взрыву экспериментов вне финансовых услуг. Есть крипто-социальные клубы, крипто-видеоигры, крипто-рестораны и даже крипто-поддерживаемые беспроводные сети.Немонетарные применения все еще довольно ограничены. Но энтузиасты крипты часто утверждают, что технология все еще молода и что для интернета потребовалось несколько десятилетий, чтобы вырасти в то, чем он является сегодня. Инвесторы вкладывают миллиарды в стартапы в крипто-области, потому что они считают, что в один прекрасный день блокчейны будут использоваться для самых разных вещей: хранения медицинских записей, отслеживания прав на потоковую музыку, а также хостинга новых социальных медиа-платформ. И крипто-экосистема привлекает множество разработчиков, что является многообещающим знаком для любой новой технологии.Вероятно вы хотите спросить, почему некоторые люди называют крипту -  пирамидой или схемой Понци и что они имеют в виду под этим?Некоторые критики считают, что рынки криптовалют в корне фальшивы, либо потому что ранние инвесторы богатеют за счет поздних инвесторов (как в схеме с классической пирамидой), либо потому что крипто-проекты заманивают ничего не подозревающих инвесторов обещаниями безопасных доходов, а затем рушатся, когда перестает поступать новые деньги (так называемая схема Понци).В мире крипты действительно полно примеров пирамид и схем Понци. Сюда входят OneCoin, мошенническая крипто-компания, которая украла 4 миллиарда долларов у инвесторов с 2014 по 2019 год; и Virgil Sigma Fund, крипто-хедж-фонд на 90 миллионов долларов, управляемый 24-летним инвестором, который признался в мошенничестве с ценными бумагами и был приговорен к семи с половиной годам тюрьмы.Но обычно критики говорят не о таких случаях. Они, скорее, утверждают, что сама по себе крипта является эксплуатационной схемой, не имеющей реальной ценности.Правы ли они?Ну, давайте попытаемся понять аргументы, которые они предлагают.В отличие от покупки акций, скажем, Apple, которая (теоретически, по крайней мере) отражает состояние здоровья основного бизнеса Apple, покупка криптовалюты больше похожа на ставку на успех идеи, говорят они. Если люди верят в Bitcoin, они покупают, и цены на Bitcoin идут вверх. Если люди перестают верить в Bitcoin, они продают, и цены на Bitcoin идут вниз.Таким образом, у владельцев крипты есть рациональный стимул убедить других людей купить ее. И если вы не считаете, что технология криптовалюты обладает внутренней ценностью, вы можете прийти к выводу, что все это напоминает пирамиду, в которой вы в основном зарабатываете деньги, вербуя других присоединиться.Но у меня ощущение, что здесь должен быть 'но'.Но! Несмотря на то, что в крипту существуют мошенничества и фальшивки, и инвесторы в крипто, безусловно, любят пытаться вербовать других людей для покупки, многие инвесторы скажут вам, что они вступают в игру, прекрасно осознавая все риски.Они уверены, что крипто-технология обладает внутренней ценностью, и что возможность хранить информацию и ценность на децентрализованном блокчейне будет привлекательной для всех видов людей и бизнесов в будущем. Они скажут вам, что они ставят на крипту как на продукт, а не как на идею — что, в каком-то смысле, не сильно отличается от покупки акций Apple, потому что вы думаете, что следующий iPhone будет популярным.Один известный инвестор, выразил мнение многих фанатов крипты, когда сказал в Twitter: 'Крипта может выглядеть как спекулятивное казино снаружи. Но это отвлекает многих от глубинной истины: казино — это троянский конь с новой финансовой системой внутри'.Можно поспорить с этой позицией, какова ценность этой 'новой финансовой системы' на самом деле. Но инвесторы в крипту явно считают, что это чего-то да стоит.Если это новая финансовая система, регулируются ли крипто-отрасль?Только слегка. В некоторых странах есть общие правила, некоторые страны приняли более строгие регулирования, а другие, например, Китай, полностью запретили торговлю криптовалютами.Но по сравнению с традиционной финансовой системой, крипта регулируется очень слабо. Мало правил, регулирующих криптоактивы, такие как 'стейблкоины' — монеты, стоимость которых привязана к государственно поддерживаемым валютам. А некоторые области крипты, такие как DeFi (децентрализованные финансы), почти полностью нерегулируемы.Отчасти это связано с тем, что сейчас все еще рано, и создание новых правил занимает время. Но это также свойство самой технологии блокчейна, большая часть которой была спроектирована таким образом, чтобы государствам было трудно ее контролировать.И я конечно же вижу вопрос в ваших глазах, сможет ли крипта заменить государственные валюты?В теории это будет обозначать, что все расчеты должны будут производиться, например,  в биткоинах или эфире и государство не сможет регулировать такие транзакции и не будет влиять на эмиссию таких криптовалют, на что государства конечно же пока не готовы пойти. Существуют также технические препятствия, которые крипта должна преодолеть, если она когда-либо собирается заменить государственную валюту. Сегодня самые популярные блокчейны — Bitcoin и Ethereum — медленные и неэффективные по сравнению с традиционными платежными сетями. (Например, блокчейн Ethereum может обрабатывать только около 15 транзакций в секунду, тогда как традиционные системы могут обрабатывать тысячи транзакций кредитных карт в секунду.)И, конечно, чтобы криптовалюта типа Bitcoin заменила государственную валюту, нужно было бы убедить миллионы или миллиарды людей использовать валюту, ценность которой сильно колеблется, не поддерживается государством и которую часто невозможно вернуть, если ее украли.И что насчет преступников? Им нравится крипта?Некоторым да. Поскольку вы можете купить и продать криптовалюту, не используя свое имя или банковский счет, крипта в свои ранние дни была идеальной для людей, которые по тем или иным причинам хотели избежать традиционной финансовой системы. Среди них были преступники, уклоняющиеся от уплаты налогов и люди, покупающие и продающие незаконные товары. Среди них были также политические диссиденты и экстремисты, некоторые из которых были заблокированы в других традиционных платежных сервисах.В результате своевременного входа в крипто-рынок, некоторые преступники разбогатели, но их количество настолько мало в текущей армии криптоэнтузиастов, что являются лишь статистической погрешностью.Другой критикой, которую я слышал, является то, что крипта вредит окружающей среде. Это правда?Это реальная проблема - и одна из самых частых возражений против крипты со стороны западных стран.Давайте начнем с того, что мы знаем наверняка. Верно, что большая часть крипто-активности сегодня происходит на блокчейнах, которые требуют больших объемов энергии для хранения и проверки транзакций. Эти сети используют механизм консенсуса 'proof-of-work' - процесс, который сравнивают с глобальной игрой в угадайку, которую проводят компьютеры, все конкурирующие за решение криптографических головоломок, чтобы добавить новую информацию в базу данных и получить награду взамен. Решение этих загадок требует мощных компьютеров, которые в свою очередь потребляют много энергии.Например, блокчейн Bitcoin использует примерно 200 тераватт-часов энергии в год. Это сопоставимо с годовым энергопотреблением Таиланда. А связанные с Bitcoin выбросы углекислого газа оцениваются примерно в 100 мегатонн в год, что сравнимо с углеродным следом Чехии.Сторонники крипты часто спорят с этими статистическими данными. Они также утверждают, что:Наша существующая финансовая система также использует много энергии, между миллионами банковских отделений, банкоматов, которые простаивают большую часть дня, золотых шахт и другой энергоемкой инфраструктуры.Многие крипто-майнинговые компьютеры уже работают от возобновляемых источников энергии, или от энергии, которая в противном случае была бы потрачена впустую.Большинство новых блокчейнов созданы с использованием механизмов консенсуса, которые требуют гораздо меньше энергии, чем proof-of-work. (Например, Ethereum перешел на новый тип механизма консенсуса под названием proof-of-stake, что уменьшило его энергопотребление на 99,5%.)И эти аргументы оправданы?Частично. Верно, что большинство новых блокчейнов разработаны таким образом, что требуют гораздо меньше энергии, чем Bitcoin, и что переход Ethereum на механизм консенсуса proof-of-stake существенно сократил его экологический след.При этом энергетические потребности Bitcoin, по ожиданиям, не уменьшатся значительно в ближайшее время. И даже если бы каждый майнер Bitcoin полностью работал на возобновляемой энергии - все равно осталась бы экологический след, связанный с поддержанием блокчейна.В целом, очевидно, что современная крипто-отрасль оказывает значительное воздействие на окружающую среду, но сложно точно измерить, насколько значительное. Многие часто цитируемые статистические данные исходят от отраслевых групп, и трудно найти достоверные, независимые данные и анализ.Но сторонники крипты в основном не спорят, что блокчейны потребляют гораздо больше энергии, чем традиционная, централизованная база данных - так же как 100 холодильников потребляют больше энергии, чем один холодильник. Они просто утверждают, что экологический ущерб от крипты со временем сократится, и что преимущества децентрализации стоят того.Некоторые сторонники крипты скажут вам, что наибольшее преимущество децентрализации - это возможность создавать валюты, приложения и виртуальные экономики, устойчивые к цензуре и контролю сверху. (Представьте версию Facebook, говорят они, в которой Марк Цукерберг не мог бы однолично решать кого заблокировать.)Другие скажут, что наибольшее преимущество децентрализации состоит в том, что она позволяет художникам и творцам более непосредственно контролировать свою экономическую судьбу, предоставляя им способ (в форме NFT и других криптоактивов) обходить посредников, таких как YouTube и Spotify, и продавать уникальные цифровые произведения напрямую своим фанатам.Самый быстрый способ начать использовать криптовалюты - это создать аккаунт на криптовалютной бирже. Она может быть связана с вашим банковским счетом и конвертировать ваши деньги в криптовалюту.При этом многие пользователи криптовалют предпочитают использовать вместо криптобирж свои собственные 'кошельки' - безопасные места для хранения криптографических ключей, которые открывают доступ к их цифровым активам.Когда у вас в кошельке есть некоторое количество криптовалюты, процесс может быть довольно простым - просто введите адрес криптовалютного кошелька получателя, оплатите комиссию за транзакцию (если это применимо) и дожидаетесь подтверждения платежа.Другие типы транзакций с криптовалютами, такие как покупка и продажа NFT, могут быть значительно сложнее, но базовый процесс отправки платежа обычно занимает всего несколько минут.И конечно возникает закономерный вопрос, почему крипта такая странная, сложная, а сообщество такое закрытое?Это, возможно, самый частый вопрос о крипту. Люди видят, как их друзья, коллеги и родственники погружаются в криптовалютную кроличью нору и через несколько дней или недель возвращаются с новым увлечением, новыми интернет-друзьями, множеством нового жаргона и кажущейся неспособностью говорить о чем-либо другом. Люди, которые верят в крипту, действительно в неё верят - до такой степени, что они могут казаться внешнему миру больше похожими на евангелистов новой религии, чем на фанатов новой технологии.Но если начать с попытки понять, что их так взволновало, возможно вам откроется много интересного."

In [175]:
t1 = fun_punctuation_text(t1)

In [176]:
t1 = fun_lemmatizing_text(t1)

In [177]:
t1 = fun_tokenize(t1)

In [178]:
text_vectorized = vectorizer.transform([t1])

In [179]:
text_vectorized.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [180]:
prediction = model.predict(text_vectorized)  
probabilities = model.predict_proba(text_vectorized)

In [181]:
print(f"Класс: {prediction[0]}")
print(f"Вероятности: {probabilities}")

Класс: 0
Вероятности: [[0.6 0.2 0.2]]
